# Topic 프로그래밍 - Python

#### Publusher Node 작성
+ #!의 부분은 이 프로그램이 어떤 언어로 쓰인지 알려주는 shebang line이다.   ##(리눅스에서만 의미를 가진다)
+ import rclpy   ##(파이썬을 통해서 작성하기 위해서 필요하다)
+ from rclpy.node import Node   ##(파이썬을 통해 ROS2를 프로그래밍 할 때 필요하다)
+ from geometry_msgs.msg import Twist   ##(geometry_msgs/msg/Twist  ⇒ from geometry_msgs.msg import Twist에서 바뀌는 형식을 잘 봐야한다.)



#### 코드 분석
+ Main문을 살펴보면 
    <pre>
    <code>

    def main(args=None):  
     rclpy.init(args=args)  
		 
		
    cmd_vel_publisher = CmdVelPublisher()
		
     rclpy.spin(cmd_vel_publisher)  

		
     cmd_vel_publisher.get_logger().info('\n==== Stop Publishing ====')  

		
     cmd_vel_publisher.destroy_node()

		
     rclpy.shutdown()으로 나타난다.
    </code>
    </pre>
    +  rclpy.init(args=args) 을 통해서 rclpy를 초기화 시켜주고 동작을 다 한 다음에는 rclpy.shutdown() 을 시켜줘야한다.
    + 새로운 node의 객체를 만드는 과정이 있고 나서도 항상 destroy시켜줘야한다.
    + 실질적으로 실행되는 node는 2줄이다.
        + rclpy.spin(cmd_vel_publisher) ##spin안에 while문이 내장되어 있으므로 spin에 node를 입력하면 주기적으로 rclpy가 node를 반복시켜준다.
	    + cmd_vel_publisher.get_logger().info('\n==== Stop Publishing ====')  

+ 특정 시간만큼만 동작시키기 위한 main을 보면

<pre>
<code>

def main(args=None):
    rclpy.init(args=args)

    cmd_vel_publisher = CmdVelPublisher()
		
    start_time = cmd_vel_publisher.get_clock().now().to_msg().sec
    clock_now  = start_time
		time_delta = 0

		
    while (clock_now - start_time) < 5:
        rclpy.spin_once(cmd_vel_publisher)
        clock_now = cmd_vel_publisher.get_clock().now().to_msg().sec

        time_delta = clock_now - start_time
        print(f'{time_delta} seconds passed')

    cmd_vel_publisher.stop_robot()

    cmd_vel_publisher.get_logger().info('\n==== Stop Publishing ====')
    cmd_vel_publisher.destroy_node()

    rclpy.shutdown()
</code>
</pre>
으로 나타난다.
   + rclpy.spin_once(cmd_vel_publisher) 전에 코드에서는 spin안에 while문이 포함되어 있다고 했는데 한번만 실행시키기 위해 있는 ROS api이다.

#### Class 내부(중요!)

+ 모든 ROS는 Composition이라는 Class를 통해 상속받아서 쓰는 방법을 권유하고 있다.
    + 여기서 상속이란 상위 Class가 구현한 것을 동일하게 사용할 수 있고 자신만의 기능도 추가할 수 있는 것을 의미한다.

<pre>
<code>
class CmdVelPublisher(Node):

    def __init__(self):
        super().__init__("cmd_vel_pub_node")
				# publisher를 생성하는 부분입니다. 하단에 추가 설명이 있습니다.
        self.publisher = self.create_publisher(Twist, "skidbot/cmd_vel", 10)

				# 어느정도의 주기로 publish 할 것인지를 선택합니다.
				# 지금의 경우 0.5초의 간격으로 publish_callback 함수를 반복 실행합니다.
        timer_period = 0.5  # seconds
        self.timer = self.create_timer(timer_period, self.publish_callback)

				# 시작한다는 log를 출력합니다.
        self.get_logger().info(" DriveForward node Started, move forward during 5 seconds \n")
    
    def publish_callback(self):
				...

    def stop_robot(self):
        ...
</code>
</pre>

+ 위의 코드에서 살펴보면 publisher의 생성은 create_publisher(사용할 message type을 지정,publish에 대응하게 되는 topic의 이름,대기열의 크기-데이터를 쌓아놓는 것이다.)를 통해서 할 수 있다.
+ Topic은 계속해서 데이터를 주고 받으므로 얼마나 자주 주고 받을 것인지 create_timer를 통해서 간격을 정해줄 수 있다.
+ publish_callback
  + 제어신호를 Topic에서 publish하기 위해서는 messsage가 필요하다.(지금 우리가 사용하는 message는 twist형식이고 twist내부에 linear.x, angular.z 처럼 데이터를 채운 형태이다.)
  <pre>
  <code>
  		def publish_callback(self):
				# 상단 형식에 맞추어 Twist Message를 채워줍니다.

				# 전방 속도 0.5 / 각속도 1.0 => 원 운동
        twist_msg = Twist()
        twist_msg.linear.x  = 0.5
        twist_msg.angular.z = 1.0
        self.publisher.publish(twist_msg)
  </code>
  </pre>
  + 즉, 위의 코드 에서 처럼 twist message를 생성하고 twist내부에 데이터를 채워주고 publish하는 과정이다.